# NRC-09 Darks: Superbias

|                                |                                                          |
|:---                            |:---                                                      |
|__CAR Title__                   | NRC-09: Darks                                            |
|__APT Program #__               | 1062                                                     |
|__NGAS #__                      | 102                                                      |
|__CAR Execution Dates(s) (UT):__| TBD                                                      |
|__JIRA Ticket Links:__          | CAR [NRCCOM-13](https://jira.stsci.edu/browse/NRCCOM-13) ||
|                                | CAP [NRCCOM-14](https://jira.stsci.edu/browse/NRCCOM-14) ||
|                                | [JSOCOPS-15](https://jira.stsci.edu/browse/JSOCOPS-15)   ||
|                                | NRCC1C-mm TBD                                            ||
|                                | NRCC1C-nn TBD                                            |
|__Analysis Team/Roles:__        | Leads: Karl Misselt (IDT), Alicia Canipe (IST)           ||
|                                | Jarron Leisenring (Analysis/Scripts)                     ||
|                                | Thomas Beatty (TSO expertise)                            ||
|                                | Bryan Hilbert (Analysis/Scripts)                         ||
|                                | Ben Sunnquist (Analysis/Scripts)                         ||
|                                | Armin Rest (Analysis/Scripts)                            ||

## Table of Contents

* [Objective](#objective)
* [Relevant links and documentation](#links)
* [Environment for analysis](#environment)
* [Imports](#imports)
* [Data for analysis](#data)
* [Superbias investigation](#superbias)

<a id='objective'></a>
## Objective

Characterize the properties of the bias using the methods described in Rest 2016 (JWST-STScI-004714): After the detector is reset before a new observation, a bias is set at the beginning of each ramp. Based on ground data, the offset is typically on the order of 10,000 ADU (differences between pixels on the order of a couple thousands of ADU). In theory, the offset won't influence the ramp-fitting, since the algorithm returns the slope and intercept for each pixel. However, the bias varies significantly between pixels, so the corrections for
IPC and cross-talk are affected. Also note that the bias has a time component; in particular, there is an initial uncertainty due to kTC noise injected with the reset. Investiage this effect. Generate new superbias reference files. Compare with ground test data. 

<a id='links'></a>
## Relevant links and documentation

|                       |                                                                                  |
|:---                   |:---                                                                              |
__JWQL dark monitor__   |                                                                             |
__NRC-09 CAR page__     |[Review Notes NRC-09](https://outerspace.stsci.edu/display/JN/Review+Notes+NRC-09)|
__NRC-09 CAP page__     |[CAP: NIRCam-09](https://outerspace.stsci.edu/display/JN/CAP%3A+NIRCam-09)        |
__Scripts__             |[NIRCam Commissioning Analysis Scripts](https://outerspace.stsci.edu/display/JN/NIRCam+Commissioning+Analysis+Scripts)                            |

<a id='environment'></a>
## Environment for analysis

Follow instructions for downloading the latest version of the pipeline to get the necessary analysis tools. Activate your environment, and then add additional tools. For this analysis, you will also need the superbias algorithm in the jwst_reffiles package, which is used to generate superbias reference files. 

Note that pipeline software is not guaranteed to work on Windows machines, but it *should* work, in theory. 

[Pipeline installation](https://github.com/spacetelescope/jwst#installing-latest-releases)

```python
conda create -n <env_name> python
conda activate <env_name>
pip install jwst
```

or for a specific version:
```python
conda create -n <env_name> python
conda activate <env_name>
pip install jwst==1.2.3
```

and add additional tools used that are not included in the ```jwst``` package:
```python
pip install ipython jupyter matplotlib pylint pandas
```

and install [jwst_reffiles](https://jwst-reffiles.readthedocs.io/en/latest/official_bad_pixel_mask.html?highlight=hot%20pixel):
```python
pip install git+https://github.com/spacetelescope/jwst_reffiles.git
```

<a id='imports'></a>
## Imports

In [ ]:
import argparse
import requests
import yaml
from glob import glob
from jwst import datamodels
from jwst.datamodels import dqflags
import numpy as np
import pandas as pd
from astropy.io import fits, ascii
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch
from jwst_reffiles.superbias import superbias
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import FuncFormatter, MaxNLocator

%matplotlib inline

In [ ]:
def show_image(data_2d, vmin, vmax, xpixel=None, ypixel=None, title=None):
    """Hilbert's function to generate a 2D, log-scaled image of the data, 
    with an option to highlight a specific pixel (with a red dot).
    
    Parameters
    ----------
    data_2d : numpy.ndarray
        Image to be displayed
        
    vmin : float
        Minimum signal value to use for scaling
        
    vmax : float
        Maximum signal value to use for scaling
        
    xpixel : int
        X-coordinate of pixel to highlight
        
    ypixel : int
        Y-coordinate of pixel to highlight
        
    title : str
        String to use for the plot title
    """
    norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                          stretch=LogStretch())
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(1, 1, 1)
    im = ax.imshow(data_2d, origin='lower', norm=norm)
    
    if xpixel and ypixel:
        plt.plot(xpixel, ypixel, marker='o', color='red', label='Selected Pixel')

    fig.colorbar(im, label='DN')
    plt.xlabel('Pixel column')
    plt.ylabel('Pixel row')
    if title:
        plt.title(title)

<a id='data'></a>
## Data for analysis

Data will be downloaded from MAST using the code written by Mario Gennaro and Armin Rest, and stored in our NIRCam data location: **TBD**

This analysis relies on ```dark``` file outputs for the dark exposures.

In [ ]:
# base_dir = '/ifs/jwst/wit/nircam/'
base_dir = '/ifs/jwst/wit/witserv/data7/nrc/'
analysis_dir = './'
ground = 'otis_long_darks'
flight = 'TBD'

In [ ]:
files = glob(base_dir+ground+'/NRC*_484_*.fits')

<a id='superbias'></a>
## Superbias investigation

The superbias algorithm is the sigma-clipped mean of all 0th frames. The superbias error is the sigma-clipped stddev of all 0th frames. Pixels with high superbias error values are flagged as UNRELIABLE_BIAS and DO_NOT_USE in the superbias DQ array. Details about the algorithm and argument options are [in the documentation](https://jwst-reffiles.readthedocs.io/en/latest/api.html?highlight=superbias#module-jwst_reffiles.superbias.superbias). 

In [ ]:
output_filename = 'superbias_jwst_reffiles.fits'

In [ ]:
superbias.make_superbias(files, clipping_sigma=3.0, max_clipping_iters=3, 
                         bad_clipping_sigma=3.0, bad_max_clipping_iters=3, 
                         bad_sigma_threshold=5.0, plot=False, save_filled=False, 
                         box_width=3, save_reffile=True, outfile=output_filename, 
                         author='jwst_reffiles', description='Super Bias Image', pedigree='GROUND', 
                         useafter='2000-01-01T00:00:00', history='', subarray=None, readpatt=None)

In [ ]:
bias = datamodels.SuperBiasModel(output_filename)

### Check how many of the pixels were flagged 

In [ ]:
do_not_use = np.where(bias.dq & dqflags.pixel['DO_NOT_USE'] > 0)
unreliable = np.where(bias.dq & dqflags.pixel['UNRELIABLE_BIAS'] > 0)

In [ ]:
num_do_not_use_flags = len(do_not_use[0])
print(('Found {} do_not_use flags.'.format(num_do_not_use_flags)))

In [ ]:
num_unreliable_flags = len(unreliable[0])
print(('Found {} unreliable flags.'.format(num_unreliable_flags)))

### Bias image 

In [ ]:
show_image(bias.data, vmin=0, vmax=5000)